<a href="https://colab.research.google.com/github/Legacy560/MTLSA25-Sales-Forecasting-Project/blob/main/model_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pylab as pl
import json
import os
import warnings
import dask.dataframe as dd
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from tqdm import tqdm
warnings.filterwarnings('ignore')
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
cd /content/gdrive/MyDrive/MLTSA25/Project

/content/gdrive/MyDrive/MLTSA25/Project


In [4]:
envs = json.load(open("kaggle.json", "r"))
os.environ["KAGGLE_USERNAME"] = envs['username']
os.environ["KAGGLE_KEY"] = envs['key']

In [5]:
# !kaggle competitions download -c m5-forecasting-accuracy

In [6]:
# !unzip m5-forecasting-accuracy.zip

In [7]:
cal = pd.read_csv('calendar.csv')
prices = pd.read_csv('sell_prices.csv')
salesVal = pd.read_csv('sales_train_validation.csv')
display(salesVal.head())
display(prices.head())
display(cal.head())


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [8]:
def downcast(df):
    """
    Optimizes memory usage of a pandas DataFrame by downcasting numeric and object columns
    to smaller data types where possible.

    Operations performed:
    - Integers are downcasted to the smallest possible int type (int8, int16, int32, int64).
    - Floats are downcasted to the smallest possible float type (float16, float32, float64).
    - Object columns are converted to 'category' if not 'date'. If the column is named 'date',
      it is converted to datetime format ('%Y-%m-%d').

    Parameters:
    -----------
    df : pandas.DataFrame
        The input DataFrame to be downcasted.

    Returns:
    --------
    df : pandas.DataFrame
        The optimized DataFrame with downcasted column types.
    memory_before : float
        Memory usage of the DataFrame (in MB) before downcasting.
    memory_after : float
        Memory usage of the DataFrame (in MB) after downcasting.
    """
    memory_before = df.memory_usage(deep=True).sum() / (1024 ** 2)  # in MB

    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i, t in enumerate(types):
        col = cols[i]
        if 'int' in str(t):
            if df[col].min() > np.iinfo(np.int8).min and df[col].max() < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif df[col].min() > np.iinfo(np.int16).min and df[col].max() < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif df[col].min() > np.iinfo(np.int32).min and df[col].max() < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)

        elif 'float' in str(t):
            if df[col].min() > np.finfo(np.float16).min and df[col].max() < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif df[col].min() > np.finfo(np.float32).min and df[col].max() < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

        elif t == object:
            if col == 'date':
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            else:
                df[col] = df[col].astype('category')

    memory_after = df.memory_usage(deep=True).sum() / (1024 ** 2)  # in MB

    print(f"Memory usage before downcasting: {memory_before:.2f} MB")
    print(f"Memory usage after downcasting : {memory_after:.2f} MB")

    return df

In [9]:
salesVal = downcast(salesVal)
print(salesVal.head())
prices = downcast(prices)
print(prices.head())
cal = downcast(cal)
print(cal.head())

Memory usage before downcasting: 456.78 MB
Memory usage after downcasting : 97.60 MB
                              id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  d_1908  \
0       CA    0    0    0    0  ...       1       3       0       1       1   
1       CA    0    0    0    0  ...       0       0       0       0       0   
2       CA    0    0    0    0  ...       2       1       2       1       1   
3       CA    0    0    0    0  ...       1       0       5       4       1   
4       CA    0    0    0    0  ...    

In [10]:
salesValLong = salesVal.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                         var_name="d", value_name="units_sold").dropna()
#This gives us a column "d" containing each sales day. This is the same as the "d" column from calendar

In [11]:
display(salesVal.head())
display(salesValLong.head(20))
display(cal.head())
display(prices.head())

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


,id,item_id,dept_id,cat_id,store_id,state_id,d,units_sold
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [12]:
#Merge with Calendar dataset
salesValLong1 = salesValLong.merge(cal, on="d", how="left")
display(salesValLong1.head())

,id,item_id,dept_id,cat_id,store_id,state_id,d,units_sold,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


In [13]:
#Merge with prices dataset
salesFull = salesValLong1.merge(prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")
display(salesFull.head(5))

,id,item_id,dept_id,cat_id,store_id,state_id,d,units_sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


# Data Splitting

In [14]:
salesFull = downcast(salesFull)

Memory usage before downcasting: 5367.24 MB
Memory usage after downcasting : 2006.46 MB


In [15]:
# Define cutoff days
last_train_day = 1913
forecast_horizon = 28
test_start_day = last_train_day + 1
test_end_day = last_train_day + forecast_horizon

# Convert day labels for filtering
train_days = [f'd_{i}' for i in range(1, last_train_day + 1)]
test_days = [f'd_{i}' for i in range(test_start_day, test_end_day + 1)]

# Split the merged salesFull dataframe
train_data = salesFull[salesFull['d'].isin(train_days)].copy()
test_data = salesFull[salesFull['d'].isin(test_days)].copy()


In [19]:
!pip uninstall -y pmdarima
!pip install numpy --upgrade
!pip install pmdarima --no-cache-dir

Found existing installation: pmdarima 2.0.4
Uninstalling pmdarima-2.0.4:
  Successfully uninstalled pmdarima-2.0.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.5 MB/s eta 0:00:00


In [22]:
def choose_d(y, p_threshold=0.05, max_d=2):
    for d in range(max_d + 1):
        y_diff = np.diff(y, n=d) if d > 0 else y
        p_val = adfuller(y_diff)[1]
        if p_val < p_threshold:
            return d
    return max_d

In [23]:
from statsmodels.tsa.stattools import acf, pacf

def choose_p_q(y_diff, nlags=20, conf=1.96):
    acf_vals = acf(y_diff, nlags=nlags)
    pacf_vals = pacf(y_diff, nlags=nlags)

    # Confidence interval (assuming white noise)
    threshold = conf / np.sqrt(len(y_diff))

    # Choose p where PACF crosses threshold
    p = next((i for i, val in enumerate(pacf_vals[1:], start=1) if abs(val) < threshold), 0)

    # Choose q where ACF crosses threshold
    q = next((i for i, val in enumerate(acf_vals[1:], start=1) if abs(val) < threshold), 0)

    return p, q


In [24]:
sample_ids = salesFull['id'].unique()[:10]  # Small sample for now

model_results = {}

for series_id in tqdm(sample_ids):
    ts = salesFull[salesFull['id'] == series_id].sort_values('d')['units_sold'].values

    # Step 1: Determine d
    d = choose_d(ts)

    # Step 2: Get differenced series
    y_diff = np.diff(ts, n=d) if d > 0 else ts

    # Step 3: Estimate p and q
    p, q = choose_p_q(y_diff)

    # Step 4: Fit ARIMA(p,d,q)
    try:
        model = ARIMA(ts, order=(p, d, q))
        fit = model.fit()
        forecast = fit.forecast(steps=28)

        model_results[series_id] = {
            'order': (p, d, q),
            'forecast': forecast.tolist()
        }

    except Exception as e:
        print(f"Failed for {series_id} with (p,d,q)=({p},{d},{q}): {e}")


100%|██████████| 10/10 [01:34<00:00,  9.47s/it]


In [25]:
model_results

{'HOBBIES_1_001_CA_1_validation': {'order': (8, 0, 0),
  'forecast': [0.16925180909505066,
   0.2601086877890045,
   0.26742485528321747,
   0.2343986858760928,
   0.20133682986143692,
   0.20652421755163947,
   0.22550105095913411,
   0.2507308996920804,
   0.26871393136847793,
   0.2737400746435325,
   0.2718350311496853,
   0.27104207373369066,
   0.2741082661110691,
   0.2800300215486711,
   0.28613697317904907,
   0.2906447955545399,
   0.2931014623762501,
   0.29443021743093833,
   0.29573285848453557,
   0.2974841208245681,
   0.29951050997032636,
   0.3014279629401271,
   0.30296590157664816,
   0.30410548882679317,
   0.3050001338317018,
   0.30581938546011805,
   0.30663719771887293,
   0.30743390626059974]},
 'HOBBIES_1_002_CA_1_validation': {'order': (4, 0, 4),
  'forecast': [0.14636081224669378,
   0.13144797200953476,
   0.22626614728384,
   0.1634617936812639,
   0.17969651129882241,
   0.24723001215683985,
   0.17184645340443971,
   0.21815422520317143,
   0.25199646582